In [ ]:
#pip install snownlp matplotlib pandas

In [ ]:
from snownlp import SnowNLP
import matplotlib.pyplot as plt
from tqdm import tqdm  # 改为纯文本模式的 tqdm，避免 Jupyter Lab widget 报错
import pandas as pd




# ——————————————————————————
# 设置起始日期，默认为 2017-01-01
# ——————————————————————————
start_date = pd.to_datetime('2020-01-01')

stkcd = '000025'

# ——————————————————————————
# 读取数据文件
# ——————————————————————————
file_path = f'/data/{stkcd}股吧帖子.csv'
df = pd.read_csv(file_path, dtype=str)

# 配置进度条和中文显示
tqdm.pandas()  # 注册 pandas 的 progress_apply
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# ——————————————————————————
# 1. 解析“更新时间”和“标题”
# ——————————————————————————
df['raw_time'] = pd.to_datetime(df['更新时间'], errors='coerce')
df['date'] = df['raw_time'].dt.normalize()

# ——————————————————————————
# 2. 删除起始日期之前的行
# ——————————————————————————
df = df[df['date'] >= start_date].reset_index(drop=True)

# ——————————————————————————
# 3. 生成情绪得分（根据标题）
# ——————————————————————————
df['sentiment_score'] = df['标题'].progress_apply(lambda x: SnowNLP(x).sentiments)

# ——————————————————————————
# 4. 按日期汇总情绪得分，取均值
# ——————————————————————————
daily_sentiment = (
    df.groupby('date', as_index=False)['sentiment_score'].mean().sort_values('date')
)

# 保存为 Excel
daily_sentiment.to_excel('daily_sentiment.xlsx', index=False)

# ——————————————————————————
# 5. 可视化情绪得分
# ——————————————————————————
plt.figure(figsize=(13, 5))
plt.plot(daily_sentiment['date'], daily_sentiment['sentiment_score'], marker='o')
plt.xticks(rotation=45)
plt.title('股吧日度情绪指标（根据标题）')
plt.xlabel('日期')
plt.ylabel('平均情绪得分 (0-1)')
plt.tight_layout()

# 保存并显示图形
plt.savefig('sentiment_plot.png')
plt.show()

print(f"Excel 文件已保存：daily_sentiment.xlsx")
print(f"情绪图形已保存：sentiment_plot.png")


In [ ]:
# ——————————————————————————
# 5. 可视化情绪得分
# ——————————————————————————
plt.figure(figsize=(13, 5))
plt.plot(daily_sentiment['date'], daily_sentiment['sentiment_score'], marker='o')
plt.xticks(rotation=45)
plt.title('股吧日度情绪指标（根据标题）')
plt.xlabel('日期')
plt.ylabel('平均情绪得分 (0-1)')
plt.tight_layout()

# 保存并显示图形
plt.savefig('sentiment_plot.png')
plt.show()

print(f"Excel 文件已保存：daily_sentiment.xlsx")
print(f"情绪图形已保存：sentiment_plot.png")
